In [6]:
import pandas as pd
import numpy as np
df = pd.read_csv('/Users/pranjal/Desktop/Causal-Inference/data/school.csv')

In [7]:
df = pd.get_dummies(df, columns = ['schid'], drop_first=True)

In [8]:
df.head()

,stuid,wattack,sfa,ppvt,sch_ppvt,schid_2,schid_3,schid_4,schid_5,schid_6,...,schid_32,schid_33,schid_34,schid_35,schid_36,schid_37,schid_38,schid_39,schid_40,schid_41
0,10158087,469,1,89,90.639175,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10217961,486,1,83,90.639175,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10486718,501,1,90,90.639175,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10540685,473,1,91,90.639175,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10750581,474,1,102,90.639175,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
outcome = 'wattack'
treatment = 'sfa'
rest = list(df.drop([outcome, treatment], axis = 1).columns)
df = df[[outcome] + [treatment] + rest]

In [10]:
#import wooldridge
#df = wooldridge.data('jtrain3')
#df['avg'] = 0.5 * (df.re74+df.re75)
#df = df.dropna()
df = df.fillna(0)
#df = df[df.avg <= 15]
y = df[outcome]
d = df[treatment]
x = df[rest].astype('float')
print(df.shape)
df.head()

(2334, 45)


,wattack,sfa,stuid,ppvt,sch_ppvt,schid_2,schid_3,schid_4,schid_5,schid_6,...,schid_32,schid_33,schid_34,schid_35,schid_36,schid_37,schid_38,schid_39,schid_40,schid_41
0,469,1,10158087,89,90.639175,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,486,1,10217961,83,90.639175,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,501,1,10486718,90,90.639175,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,473,1,10540685,91,90.639175,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,474,1,10750581,102,90.639175,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Simple Comparision of Means
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:                wattack   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     76.52
Date:                Mon, 05 Dec 2022   Prob (F-statistic):           4.09e-18
Time:                        01:16:47   Log-Likelihood:                -10251.
No. Observations:                2334   AIC:                         2.051e+04
Df Residuals:                    2332   BIC:                         2.052e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             7.0914      0.811      8.748      0.0

In [11]:
# Pooled Regression Adjustment
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d, x], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:               post_ach   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     4.278
Date:                Mon, 05 Dec 2022   Prob (F-statistic):             0.0144
Time:                        01:01:48   Log-Likelihood:                -2274.2
No. Observations:                 521   AIC:                             4554.
Df Residuals:                     518   BIC:                             4567.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             4.9337      1.688      2.923      0.0

In [15]:
import numpy as np
from doubleml.datasets import make_plr_CCDDHNR2018
from doubleml import DoubleMLData

np.random.seed(1234)
dml_data_bonus = DoubleMLData(df, y_col=outcome,
                                  d_cols=treatment,
                                  x_cols=list(rest))
print(dml_data_bonus)
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
learner = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 6)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)
learner = LassoCV()
ml_l_sim = clone(learner)
ml_m_sim = clone(learner)
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

from doubleml import DoubleMLPLR
np.random.seed(3141)
obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: wattack
Treatment variable(s): ['sfa']
Covariates: ['stuid', 'ppvt', 'sch_ppvt', 'schid_2', 'schid_3', 'schid_4', 'schid_5', 'schid_6', 'schid_7', 'schid_8', 'schid_9', 'schid_10', 'schid_11', 'schid_12', 'schid_13', 'schid_14', 'schid_15', 'schid_16', 'schid_17', 'schid_18', 'schid_19', 'schid_20', 'schid_21', 'schid_22', 'schid_23', 'schid_24', 'schid_25', 'schid_26', 'schid_27', 'schid_28', 'schid_29', 'schid_30', 'schid_31', 'schid_32', 'schid_33', 'schid_34', 'schid_35', 'schid_36', 'schid_37', 'schid_38', 'schid_39', 'schid_40', 'schid_41']
Instrument variable(s): None
No. Observations: 2334

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2334 entries, 0 to 2333
Columns: 45 entries, wattack to schid_41
dtypes: float64(1), int64(4), uint8(40)
memory usage: 182.5 KB

================== DoubleM

In [16]:
import numpy as np
from doubleml.datasets import make_plr_CCDDHNR2018
from doubleml import DoubleMLData

np.random.seed(1234)
dml_data_bonus = DoubleMLData(df, y_col=outcome,
                                  d_cols=treatment,
                                  x_cols=list(rest))
print(dml_data_bonus)
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
learner = RandomForestRegressor(n_estimators = 100, max_features = 'sqrt', max_depth= 6)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)
learner = LassoCV()
ml_l_sim = clone(learner)
ml_m_sim = clone(learner)
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

from doubleml import DoubleMLPLR
np.random.seed(3141)
obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: wattack
Treatment variable(s): ['sfa']
Covariates: ['stuid', 'ppvt', 'sch_ppvt', 'schid_2', 'schid_3', 'schid_4', 'schid_5', 'schid_6', 'schid_7', 'schid_8', 'schid_9', 'schid_10', 'schid_11', 'schid_12', 'schid_13', 'schid_14', 'schid_15', 'schid_16', 'schid_17', 'schid_18', 'schid_19', 'schid_20', 'schid_21', 'schid_22', 'schid_23', 'schid_24', 'schid_25', 'schid_26', 'schid_27', 'schid_28', 'schid_29', 'schid_30', 'schid_31', 'schid_32', 'schid_33', 'schid_34', 'schid_35', 'schid_36', 'schid_37', 'schid_38', 'schid_39', 'schid_40', 'schid_41']
Instrument variable(s): None
No. Observations: 2334

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2334 entries, 0 to 2333
Columns: 45 entries, wattack to schid_41
dtypes: float64(1), int64(4), uint8(40)
memory usage: 182.5 KB

================== DoubleM

In [17]:
# DML regression - still yeilds unbiased estimate of ATE 
from econml.dml import LinearDML
est = LinearDML(random_state=45)
est.fit(y, d, X=None,W=x)
est.summary()

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,6.468,0.808,8.009,0.0,4.885,8.051
